<a href="https://colab.research.google.com/github/TanmayKhot/AI_project/blob/main/1_Baseline_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reverse Image Search

Reverse image search essentially also known as instance retrieval, enables us to build scenarios beyond just keywork search. Thus, here given an image as a query we output some other images that are similar and correlated to the query image. 

Since, we need to search among millions of images to find other similar images we ideally need to summarize the information contained in the millions of pixels in a image into a smaller representation. Deep neural networks now come into play. Using a Convolutional Neural Network an input image is converted into a feature vector of thousand dimensions, which is then supplied as an input query and returns the top matches for the query image.

These feature vectors are also called as embeddings which is a collection of floating point values. Embeddings represent the most important and salient constituents of the image. 

Now to find similar images we pass the images through a pretrained convolutional neural network, ResNet-50 to extract feature embeddings. This is not optimal as res-net50 is not specilized in recognizing faces.

In [ ]:
# Importing required libraries
import json
import tqdm
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from multiprocessing import cpu_count
from tqdm.contrib.concurrent import process_map
import urllib.request
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from numpy.linalg import norm

#### Dataset

Labeled Faces in Wild (LFW) is the dataset we use for the project. LFW contains face photographs where each face has been labeled with the name of the person pictured. The data set contains more than 13,000 images of faces collected from the web.

There are 4 different sets of LFW images. For our project, we work with the 'Deep Funneled' images. Deep funneled LFW dataset is formed by a combination of unsupervised joint alignment with unsupervised feature learning. 

We incorporate the deep funneled LFW dataset as it produces superior results over the original dataset.

## Extract data

After having manually uploaded the .tgz file containing the data we need to unzip it 

In [ ]:
!tar -xf lfw-deepfunneled.tgz

^C


In [ ]:
#function to get path from each folder recursively 
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1
    return file_list

In [ ]:
#provide path to main folder of dataset and get img paths to the datasets
root_dir = os.getcwd()+ '/lfw-deepfunneled/'
filenames = sorted(get_file_list(root_dir))

In [ ]:
#see it list of paths
filenames[:3]

['/home/sb8389/A Projects/Reverse Image Project/lfw-deepfunneled/AJ_Cook/AJ_Cook_0001.jpg',
 '/home/sb8389/A Projects/Reverse Image Project/lfw-deepfunneled/AJ_Lamas/AJ_Lamas_0001.jpg',
 '/home/sb8389/A Projects/Reverse Image Project/lfw-deepfunneled/Aaron_Eckhart/Aaron_Eckhart_0001.jpg']

## Prepare Model

A deep learning model trained on a large set of labelled images have basically become an automatic “feature extractor”. This means that when analyzing our images, we basically get as output a “feature vector” which contains relevant information about the image content. Training a model from scratch can take up a lot of computing resources and time. Leveraging the concept of transfer learning helps us get a headstart into the training process. Transfer learning involves using models trained on one problem as a starting point on a related problem. Transfer learning has the benefit of decreasing the training time for a neural network model and can result in lower generalization error.

**Residual Network (ResNet50):**

For our project we use pre-trained weights from ResNet 50 as the baseline model to generate embeddings. ResNet is a CNN that is 50 layers deep. The pretrained version of the network is trained on more than a million images from the ImageNet database. The architecture is as follows:

<img src = 'https://drive.google.com/uc?id=1Gm6UQVJ7mvqyuW6aHoD-kh8PW21EGCCz'>

ResNet ideally applies identity mapping, meaning that the input to some layer is passed directly or as a shortcut to some other layer. It works on the principle of skip connection. Skip connection is basically the identity mapping where the input from previous layer is added directly to the output of the other layer. 

We load ResNet-50 without the top classification layers, so as to get the feature embeddings only. The 'extract_features' function below performs the following operations:
1. Load image from the image path
2. Resize to the format supported by ResNet-50
3. Pre-process the image
4. Extract the features
5. Normalize the features

In [ ]:
# Set the channel first for better performance
from tensorflow.keras import backend
backend.set_image_data_format('channels_last')
print(backend.image_data_format())

channels_last


In [ ]:
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False,input_shape=(224, 224,3),pooling='avg')

In [ ]:
#Creating the directory structure
dirName = 'model'
if not os.path.exists(dirName):
    os.makedirs(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")   

Directory  model  Created 


In [ ]:
#Save the model in SavedModel format
model.save('./model/', save_format='tf')

INFO:tensorflow:Assets written to: ./model/assets


In [ ]:
# takes an image path, loads the image, resizes it to proper 
#dimensions supported by ResNet-50, extracts the features, and then normalizes them
def extract_features(img_path, model):
    input_shape = (224, 224,3)
    img = image.load_img(img_path, target_size=(
        input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = tf.keras.applications.resnet50.preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

In [ ]:
#let's try with one image
features = extract_features('/home/sb8389/A Projects/Reverse Image Project/lfw-deepfunneled/AJ_Cook/AJ_Cook_0001.jpg', model)
print(len(features))

2048


In [ ]:
#get features for each pic from model
feature_list = []
for i in tqdm_notebook(range(len(filenames))):
    feature_list.append(extract_features(filenames[i], model))

/usr/local/stow/python-3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pickle
pickle.dump(feature_list, open('saved/features-lfw-deepfunneled-resnet50.pickle', 'wb'))
pickle.dump(filenames, open('saved/filenames-lfw-deepfunneled.pickle','wb'))

Thus, we now have the embeddings from our ResNet-50 baseline model which we will upload to EC2.